In [1]:
from bs4 import BeautifulSoup # For HTML parsing
from time import sleep # To prevent overwhelming the server between connections
import pandas as pd # For converting results to a dataframe and bar chart plots
# For Visualizations
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
%matplotlib inline

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://condos.ca')
## You will need to create an account and login 


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Driver [C:\Users\perry\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [11]:
#For simplicity, this scrapes downtown toronto condos that were sold in 2020 only.
dt_sold_prices=[]
dt_sold_location=[]
dt_sold_sqft=[]
dt_sold_bd=[]
dt_sold_all=[]
dt_sold_date=[]

def get_page(page):
    #The URL below already filters for downtown toronto condos only that are at least 300 sq ft,that were sold in 2020
    url= f'https://condos.ca/toronto/condos-for-sale?size_range=300%2C999999999&property_type=Condo%20Apt%2CComm%20Element%20Condo%2CLeasehold%20Condo&mode=Sold&end_date_unix=exact%2C2020&sublocality_id=14&page={page}'
    driver.get(url) 
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    return soup

#attempt to scrape 12 pages.
for page in range(12):
    soup = get_page(page)
    
    #Scrape prices
    for tag in soup.find_all('div',class_ = 'styles___AskingPrice-sc-54qk44-4 styles___ClosePrice-sc-54qk44-5 dHPUdq hwkkXU'):
        dt_sold_prices.append(tag.get_text())

    #scrape location
    for tag in soup.find_all('address',class_ = 'styles___Address-sc-54qk44-13 gTwVlm'):
        dt_sold_location.append(tag.get_text())

    #scrape sq ft
    for tag in soup.find_all(class_ = 'styles___Size-sc-54qk44-8 KRKbD'):
        dt_sold_sqft.append(tag.get_text())

    #scrape bedrooms    
    for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
        bedrooms = tag.get_text()[0:3]
        dt_sold_bd.append(bedrooms)
        
    #scrape all other containers that includes parking
    for tag in soup.find_all(class_ = 'styles___InfoHolder-sc-54qk44-7 jtFhfz'):
        everything = tag.get_text()[0:]
        dt_sold_all.append(everything)

    #scrape sold date
    for tag in soup.find_all(class_ = 'styles___Banner-sc-1a2baz3-0 iWaXsf'):
        everything = tag.get_text()[0:]
        dt_sold_date.append(everything)

In [12]:
#combine all scrape lists into a dictionary so that it cane be transformed to a dataframe
a = {'Prices':dt_sold_prices,
                         'Location':dt_sold_location,
                         'Square Ft':dt_sold_sqft,
                         'Bedrooms':dt_sold_bd,
                         'Everything':dt_sold_all,
                         'Sold Date':dt_sold_date}

df_sold_condos = pd.DataFrame.from_dict(a, orient='index')
df_sold_condos = df_sold_condos.transpose()

In [13]:
#add a column that tags the Downtown region
df_sold_condos['Region']= 'Downtown'
df_sold_condos

# !!! There should be 48 listings per page * 11 pages = ~528 records, but it's not picking up everything. 
#Sometimes it scrapes even less than 213 rows. Not consistent when trying to apply to other regions/years.

,Prices,Location,Square Ft,Bedrooms,Everything,Sold Date,Region
0,"$570,000 $569,000",606 - 39 Sherbourne St,600-699 sqft,1+1,1+1BD1BA0 Parking600-699 sqft,Sold in 2020,Downtown
1,"$570,000 $574,900",319 - 33 Mill St,600-699 sqft,1+1,1+1BD1BA0 Parking600-699 sqft,Sold in 2020,Downtown
2,"$630,000 $650,000",1403 - 15 Grenville St,500-599 sqft,1BD,1BD1BA0 Parking500-599 sqft,Sold in 2020,Downtown
3,"$680,888 $699,900",LPH25 - 1030 King St W,700-799 sqft,1+1,1+1BD2BA0 Parking700-799 sqft,Sold in 2020,Downtown
4,"$695,000 $699,990",1504 - 208 Queens Quay W,700-799 sqft,1+1,1+1BD2BA1 Parking700-799 sqft,Sold in 2020,Downtown
...,...,...,...,...,...,...,...
208,"$495,000 $499,000",408 - 109 Front St E,500-599 sqft,1BD,1BD1BA0 Parking500-599 sqft,Sold in 2020,Downtown
209,"$575,000 $559,900",1219 - 1 Shaw St,600-699 sqft,1BD,1BD1BA1 Parking600-699 sqft,Sold in 2020,Downtown
210,"$525,000 $559,900",416 - 350 Wellington St W,600-699 sqft,1BD,1BD1BA0 Parking600-699 sqft,Sold in 2020,Downtown
211,"$602,000 $599,000",3508 - 21 Iceboat Terr,700-799 sqft,1+1,1+1BD1BA1 Parking700-799 sqft,Sold Conditional in 2020,Downtown
